In [1]:
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.feature_selection import SelectKBest

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

The goal here is to build and evalutate simple models based on mean meter readings (per day of week, per hour).

In [5]:
df_meters = pd.read_csv('../../data/intermediate/building_1176/train_and_building_meta_1176.csv',
                        usecols=['timestamp', 'building_id', 'meter', 'meter_reading'],
                        parse_dates=['timestamp'], index_col=['timestamp'])
df_meters.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26345 entries, 2016-01-01 00:00:00 to 2016-12-31 23:00:00
Data columns (total 3 columns):
building_id      26345 non-null int64
meter            26345 non-null int64
meter_reading    26345 non-null float64
dtypes: float64(1), int64(2)
memory usage: 823.3 KB


In [6]:
df_meters.head()

,building_id,meter,meter_reading
timestamp,,,
2016-01-01 00:00:00,1176,0,70.0620
2016-01-01 00:00:00,1176,1,13.6612
2016-01-01 00:00:00,1176,2,0.0000
2016-01-01 01:00:00,1176,0,70.0620
2016-01-01 01:00:00,1176,1,13.6612


In [7]:
# Check all building ids are 1176 then drop column 'building_id'
print('building_id values : {}'.format(df_meters.building_id.unique()))
df_meters.drop('building_id', axis=1, inplace=True)
df_meters.head()

building_id values : [1176]


,meter,meter_reading
timestamp,,
2016-01-01 00:00:00,0,70.0620
2016-01-01 00:00:00,1,13.6612
2016-01-01 00:00:00,2,0.0000
2016-01-01 01:00:00,0,70.0620
2016-01-01 01:00:00,1,13.6612


In [10]:
# keep only electricity meter

df_meter = df_meters[df_meters['meter']==0]
df_meter.head()

,meter,meter_reading
timestamp,,
2016-01-01 00:00:00,0,70.062
2016-01-01 01:00:00,0,70.062
2016-01-01 02:00:00,0,69.186
2016-01-01 03:00:00,0,68.678
2016-01-01 04:00:00,0,69.113


In [11]:
df_meter.shape

(8777, 2)

An important and tricky point is how are we going to split the data between train and validation.<br>
For time-serie data, usually we cannot do the usual random split, because of correlations.<br>
Usually, some kind of walk-forward approach is used.